In [1]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

from sklearn import preprocessing, pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from cfmining.mip_algorithms import LinearRecourseActions, LinearRecourseActionsMulti
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier, GeneralClassifier, TreeClassifier, LinearClassifier
from cfmining.action_set import ActionSet

Using Python-MIP package version 1.7.2


In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

## Preparing dataset

In [3]:
giveme_df = pd.read_csv('data/givemecredit/cs-training.csv', index_col=0).dropna()
y = giveme_df['SeriousDlqin2yrs']
X = giveme_df.drop('SeriousDlqin2yrs', axis=1)

In [4]:
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=100)

## Setting ActionSet base parameters

In [5]:
action_set_base = ActionSet(X = X)
for feat in action_set_base:
    if feat.name in ['age', 'MonthlyIncome']:
        feat.mutable = False
        #feat.flip_direction = 1
        feat.step_direction = -1
    if feat.name in ['NumberOfDependents']:
        feat.mutable = False
        #feat.flip_direction = 1
        feat.step_direction = 1
    if feat.name in ['RevolvingUtilizationOfUnsecuredLines', 'DebtRatio']:
        feat.mutable = True
        #feat.flip_direction = -1
        feat.step_direction = -1
    if feat.name in ['NumberOfTime30-59DaysPastDueNotWorse',
                     'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
                     'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse']:
        feat.mutable = True
        #feat.flip_direction = -1
        feat.step_direction = 1
    #print(feat.name, feat.step_direction)
    feat.update_grid()

## Logistic regression model

In [6]:
stand = preprocessing.StandardScaler()
clf_base = LogisticRegression(max_iter=1000, class_weight='balanced')
clf = pipeline.Pipeline([('std', stand), ('clf', clf_base)])

grid = GridSearchCV(
  clf, param_grid={'clf__C': np.logspace(-4, 3)},
  cv=5,
  scoring='roc_auc',
  verbose=0
)

grid.fit(Xtr, ytr)
clf_lgr = grid.best_estimator_

In [7]:
clf_lgr_mapocam = MonotoneClassifier(clf_lgr, X=Xtr, y=ytr, threshold=0.8)
coefficients = clf_lgr['clf'].coef_[0]/clf_lgr['std'].scale_
intercept = clf_lgr['clf'].intercept_[0]-coefficients@clf_lgr['std'].mean_

/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


## Preparing counterfactual parameters

In [8]:
action_set = copy.deepcopy(action_set_base)

print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set]))

ActionSet stats
Number of actionable features: 7
Mean number of actions per feature: 40.3
Max number of actions per feature: 100


In [9]:
scores = pd.Series(clf_lgr.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < .8].index

In [10]:
percCalc = PercentileCalculator(action_set=action_set)

## Linear programming

In [11]:
lp_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActions(action_set, individual, coefficients, intercept, 0.8)
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_lgr_results[i] = {}
    lp_lgr_results[i]['enum'] = lp
    lp_lgr_results[i]['indiv'] = individual
    lp_lgr_results[i]['solution'] = [int(s) if act.variable_type=='int' else float(s) for s, act in zip(lp.solution, action_set)]
    lp_lgr_results[i]['stats'] = lp.stats
    lp_lgr_results[i]['recourse'] = lp.recourse
    lp_lgr_results[i]['time'] = lp_time
    lp_lgr_results[i]['cost'] = criteria.f(lp_lgr_results[i]['solution']) if ~any(np.isnan(lp_lgr_results[i]['solution'])) else float('inf')

100%|██████████| 97/97 [00:03<00:00, 26.16it/s]


## Greedy approach

In [12]:
greedy_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    greedy = Greedy(action_set, individual, clf_lgr_mapocam, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_lgr_results[i] = {}
    greedy_lgr_results[i]['solution'] = greedy.solution
    greedy_lgr_results[i]['time'] = greedy_time
    greedy_lgr_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

100%|██████████| 97/97 [00:05<00:00, 16.75it/s]


## MAPOCAM

In [13]:
mapocam_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_lgr_results[i] = {}
    mapocam_lgr_results[i]['enum'] = mapocam
    mapocam_lgr_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_lgr_results[i]['time'] = mapocam_time
    mapocam_lgr_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 97/97 [00:01<00:00, 77.97it/s]


## Performance comparison for single objective


In [14]:
lp_lgr_costs = np.array([lp_lgr_results[i]['cost'] for i in denied_individuals])
greedy_lgr_costs = np.array([greedy_lgr_results[i]['cost'] for i in denied_individuals])
mapocam_lgr_costs = np.array([mapocam_lgr_results[i]['cost'] for i in denied_individuals])

lp_lgr_time = np.array([lp_lgr_results[i]['time'] for i in denied_individuals])
greedy_lgr_time = np.array([greedy_lgr_results[i]['time'] for i in denied_individuals])
mapocam_lgr_time = np.array([mapocam_lgr_results[i]['time'] for i in denied_individuals])

In [15]:
print('Time performance')
print('Ustun:', np.mean(lp_lgr_time))
print('Greedy:', np.mean(greedy_lgr_time))
print('Proposed:', np.mean(mapocam_lgr_time))

Time performance
Ustun: 0.02225900863863758
Greedy: 0.059070602097769376
Proposed: 0.012482646881535496


In [16]:
print('Best solution rate')
best_costs = np.min([lp_lgr_costs, greedy_lgr_costs, mapocam_lgr_costs], axis=0)
print('Ustun:', np.mean(lp_lgr_costs==best_costs))
print('Greedy:', np.mean(greedy_lgr_costs==best_costs))
print('Proposed:', np.mean(mapocam_lgr_costs==best_costs))

Best solution rate
Ustun: 1.0
Greedy: 0.14432989690721648
Proposed: 0.9896907216494846


## Comparing Pareto enumeration

## Percentile vs changes

In [17]:
mapocam_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_pc[i] = {}
    mapocam_results_pc[i]['enum'] = mapocam
    mapocam_results_pc[i]['time'] = mapocam_time
    mapocam_results_pc[i]['num'] = len(mapocam.solutions)

100%|██████████| 97/97 [00:02<00:00, 47.57it/s]


In [18]:
lp_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    0.8, max_changes=3, enumeration_type='remove_number_actions',
                                    compare=PercentileChangesCriterion(individual, percCalc))
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_pc[i] = {}
    lp_results_pc[i]['enum'] = lp
    lp_results_pc[i]['time'] = lp_time
    lp_results_pc[i]['num'] = len(lp.solutions)

100%|██████████| 97/97 [00:01<00:00, 49.70it/s]


In [19]:
bf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_pc[i] = {}
    bf_results_pc[i]['enum'] = bf
    bf_results_pc[i]['time'] = bf_time
    bf_results_pc[i]['num'] = len(bf.solutions)

100%|██████████| 97/97 [1:32:59<00:00, 57.52s/it]    


## Features

In [20]:
mapocam_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_feat[i] = {}
    mapocam_results_feat[i]['enum'] = mapocam
    mapocam_results_feat[i]['time'] = mapocam_time
    mapocam_results_feat[i]['num'] = len(mapocam.solutions)

100%|██████████| 97/97 [00:01<00:00, 70.02it/s]


In [21]:
lp_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    0.8, max_changes=3, enumeration_type='remove_dominated')
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_feat[i] = {}
    lp_results_feat[i]['enum'] = lp
    lp_results_feat[i]['time'] = lp_time
    lp_results_feat[i]['num'] = len(lp.solutions)

100%|██████████| 97/97 [00:05<00:00, 16.66it/s]


In [22]:
bf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_feat[i] = {}
    bf_results_feat[i]['enum'] = bf
    bf_results_feat[i]['time'] = bf_time
    bf_results_feat[i]['num'] = len(bf.solutions)

100%|██████████| 97/97 [24:25<00:00, 15.11s/it]  


In [23]:
brute_sols_pc = np.array([bf_results_pc[i]['num'] for i in denied_individuals])
mapocam_sols_pc = np.array([mapocam_results_pc[i]['num'] for i in denied_individuals])
lp_sols_pc = np.array([lp_results_pc[i]['num'] for i in denied_individuals])

brute_sols_feat = np.array([bf_results_feat[i]['num'] for i in denied_individuals])
mapocam_sols_feat = np.array([mapocam_results_feat[i]['num'] for i in denied_individuals])
lp_sols_feat = np.array([lp_results_feat[i]['num'] for i in denied_individuals])

brute_time_pc = np.array([bf_results_pc[i]['time'] for i in denied_individuals])
mapocam_time_pc = np.array([mapocam_results_pc[i]['time'] for i in denied_individuals])
lp_time_pc = np.array([lp_results_pc[i]['time'] for i in denied_individuals])

brute_time_feat = np.array([bf_results_feat[i]['time'] for i in denied_individuals])
mapocam_time_feat = np.array([mapocam_results_feat[i]['time'] for i in denied_individuals])
lp_time_feat = np.array([lp_results_feat[i]['time'] for i in denied_individuals])

## Testing same number of solutions


In [24]:
print('Changes vs percentile:', np.mean(brute_sols_pc==mapocam_sols_pc)==1, np.mean(lp_sols_pc==mapocam_sols_pc)==1)
print('Features:', np.mean(brute_sols_feat==mapocam_sols_feat)==1, np.mean(lp_sols_feat==mapocam_sols_feat)==1)

Changes vs percentile: True True
Features: True True


In [25]:
print('Changes vs percentile:')
print('Ustun:',np.mean(lp_time_pc))
print('Brute force:',np.mean(brute_time_pc))
print('Proposed:', np.mean(mapocam_time_pc))
print('Features:')
print('Ustun:',np.mean(lp_time_feat))
print('Brute force:',np.mean(brute_time_feat))
print('Proposed:', np.mean(mapocam_time_feat))

Changes vs percentile:
Ustun: 0.019725759456068585
Brute force: 57.51450755426026
Proposed: 0.020601223866196023
Features:
Ustun: 0.059441892967857034
Brute force: 15.107013488274797
Proposed: 0.014030452011172305


## Random forest

In [26]:
from sklearn import tree, ensemble
from cfmining.mip_algorithms import ForestRecourseActions
from cfmining.predictors import TreeClassifier

In [27]:
clf_rt = ensemble.RandomForestClassifier(n_estimators=5, max_depth=5, max_leaf_nodes=31, class_weight='balanced_subsample')
clf_rt.fit(Xtr, ytr);

## Preparing counterfactual parameters

In [28]:
action_set_tree = copy.deepcopy(action_set_base)
action_set_tree.embed_forest(clf_rt)
for action in action_set_tree:
    action.flip_direction = action.step_direction
    
print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_tree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_tree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_tree]))

ActionSet stats
Number of actionable features: 7
Mean number of actions per feature: 10.0
Max number of actions per feature: 26


In [29]:
scores = pd.Series(clf_rt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < .8].index

In [30]:
percCalc = PercentileCalculator(action_set=action_set_tree)

In [31]:
mapocam_clf_rf = TreeClassifier(clf_rt, Xtr, ytr, threshold=0.8)
mapocam_clf_rf_fast = TreeClassifier(clf_rt, Xtr, ytr, threshold=0.8, use_predict_max=True)

## Linear programming

In [32]:
lp_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=0.8, max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_rf_results[i] = {}
    lp_rf_results[i]['enum'] = lp
    lp_rf_results[i]['indiv'] = individual
    lp_rf_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_rf_results[i]['time'] = lp_time
    lp_rf_results[i]['cost'] = criteria.f(lp_rf_results[i]['solution']) if not any(np.isnan(lp_rf_results[i]['solution'])) else float('inf')

  1%|          | 1/90 [00:00<00:53,  1.66it/s]

gurobi version 9.0 found


100%|██████████| 90/90 [00:02<00:00, 38.16it/s]


## Greedy approach

In [33]:
greedy_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    greedy = Greedy(action_set_tree, individual, mapocam_clf_rf_fast, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_rf_results[i] = {}
    greedy_rf_results[i]['solution'] = greedy.solution
    greedy_rf_results[i]['time'] = greedy_time
    greedy_rf_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

100%|██████████| 90/90 [00:01<00:00, 66.69it/s]


## MAPOCAM

In [34]:
mapocam_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results[i] = {}
    mapocam_rf_results[i]['enum'] = mapocam
    mapocam_rf_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results[i]['time'] = mapocam_time
    mapocam_rf_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 90/90 [00:03<00:00, 28.76it/s]


In [35]:
mapocam_rf_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_plus[i] = {}
    mapocam_rf_results_plus[i]['enum'] = mapocam
    mapocam_rf_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results_plus[i]['time'] = mapocam_time
    mapocam_rf_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 90/90 [00:00<00:00, 102.98it/s]


In [36]:
lp_rf_costs = np.array([lp_rf_results[i]['cost'] for i in denied_individuals])
greedy_rf_costs = np.array([greedy_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs = np.array([mapocam_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs_plus = np.array([mapocam_rf_results_plus[i]['cost'] for i in denied_individuals])

lp_rf_time = np.array([lp_rf_results[i]['time'] for i in denied_individuals])
greedy_rf_time = np.array([greedy_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time = np.array([mapocam_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time_plus = np.array([mapocam_rf_results_plus[i]['time'] for i in denied_individuals])

In [37]:
print('Time performance')
print('Ustun:', np.mean(lp_rf_time))
print('Greedy:', np.mean(greedy_rf_time))
print('Proposed:', np.mean(mapocam_rf_time))
print('Proposed plus:', np.mean(mapocam_rf_time_plus))

Time performance
Ustun: 0.02040881991593374
Greedy: 0.014715407954321967
Proposed: 0.034417643967188064
Proposed plus: 0.009475807069490354


In [38]:
print('Best solution rate')
best_costs = np.min([lp_rf_costs, greedy_rf_costs, mapocam_rf_costs, mapocam_rf_costs_plus], axis=0)
print('Ustun:', np.mean(lp_rf_costs==best_costs))
print('Greedy:', np.mean(greedy_rf_costs==best_costs))
print('Proposed:', np.mean(mapocam_rf_costs==best_costs))
print('Proposed plus:', np.mean(mapocam_rf_costs_plus==best_costs))

Best solution rate
Ustun: 1.0
Greedy: 0.8444444444444444
Proposed: 1.0
Proposed plus: 1.0


## Comparing Pareto enumeration

## Percentile vs changes

In [39]:
mapocam_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc[i] = {}
    mapocam_rf_results_pc[i]['enum'] = mapocam
    mapocam_rf_results_pc[i]['time'] = mapocam_time
    mapocam_rf_results_pc[i]['num'] = len(mapocam.solutions)

100%|██████████| 90/90 [00:09<00:00,  9.65it/s]


In [40]:
mapocam_rf_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc_plus[i] = {}
    mapocam_rf_results_pc_plus[i]['enum'] = mapocam
    mapocam_rf_results_pc_plus[i]['time'] = mapocam_time
    mapocam_rf_results_pc_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 90/90 [00:00<00:00, 120.76it/s]


In [41]:
lp_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=0.6,
                               max_changes=3, multi_solution=True,
                               cost_type='linear', compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_pc[i] = {}
    lp_rf_results_pc[i]['enum'] = lp
    lp_rf_results_pc[i]['time'] = lp_time
    lp_rf_results_pc[i]['num'] = len(lp.solutions)

100%|██████████| 90/90 [00:02<00:00, 33.38it/s]


In [42]:
bf_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_pc[i] = {}
    bf_rf_results_pc[i]['enum'] = bf
    bf_rf_results_pc[i]['time'] = bf_time
    bf_rf_results_pc[i]['num'] = len(bf.solutions)

100%|██████████| 90/90 [00:57<00:00,  1.56it/s]


## Features

In [43]:
mapocam_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat[i] = {}
    mapocam_rf_results_feat[i]['enum'] = mapocam
    mapocam_rf_results_feat[i]['time'] = mapocam_time
    mapocam_rf_results_feat[i]['num'] = len(mapocam.solutions)

100%|██████████| 90/90 [00:24<00:00,  3.65it/s]


In [44]:
mapocam_clf_rf_fast = TreeClassifier(clf_rt, Xtr, ytr, threshold=0.8, use_predict_max=True)

In [45]:
mapocam_rf_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat_plus[i] = {}
    mapocam_rf_results_feat_plus[i]['enum'] = mapocam
    mapocam_rf_results_feat_plus[i]['time'] = mapocam_time
    mapocam_rf_results_feat_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 90/90 [00:01<00:00, 62.39it/s]


In [46]:
mapocam_rf_time_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['time'] for i in denied_individuals])
print('Proposed:', np.mean(mapocam_rf_time_feat_plus))

Proposed: 0.01578041497204039


In [47]:
lp_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=0.8,
                               max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_feat[i] = {}
    lp_rf_results_feat[i]['enum'] = lp
    lp_rf_results_feat[i]['time'] = lp_time
    lp_rf_results_feat[i]['num'] = len(lp.solutions)

100%|██████████| 90/90 [00:02<00:00, 35.08it/s]


In [48]:
bf_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_feat[i] = {}
    bf_rf_results_feat[i]['enum'] = bf
    bf_rf_results_feat[i]['time'] = bf_time
    bf_rf_results_feat[i]['num'] = len(bf.solutions)

100%|██████████| 90/90 [00:43<00:00,  2.09it/s]


In [49]:
brute_rf_sols_pc = np.array([bf_rf_results_pc[i]['num'] for i in denied_individuals])
lp_rf_sols_pc = np.array([lp_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc = np.array([mapocam_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['num'] for i in denied_individuals])

brute_rf_sols_feat = np.array([bf_rf_results_feat[i]['num'] for i in denied_individuals])
lp_rf_sols_feat = np.array([lp_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat = np.array([mapocam_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['num'] for i in denied_individuals])

brute_rf_time_pc = np.array([bf_rf_results_pc[i]['time'] for i in denied_individuals])
lp_rf_time_pc = np.array([lp_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc = np.array([mapocam_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['time'] for i in denied_individuals])

brute_rf_time_feat = np.array([bf_rf_results_feat[i]['time'] for i in denied_individuals])
lp_rf_time_feat = np.array([lp_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat = np.array([mapocam_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['time'] for i in denied_individuals])

In [50]:
print('Changes vs percentile:', np.mean(brute_rf_sols_pc==mapocam_rf_sols_pc)==1, np.mean(lp_rf_sols_pc==mapocam_rf_sols_pc)==1, np.mean(mapocam_rf_sols_pc_plus==mapocam_rf_sols_pc)==1)
print('Features:', np.mean(brute_rf_sols_feat==mapocam_rf_sols_feat)==1, np.mean(lp_rf_sols_feat==mapocam_rf_sols_feat)==1, np.mean(mapocam_rf_sols_feat_plus==mapocam_rf_sols_feat)==1)

Changes vs percentile: True False True
Features: True True True


In [51]:
print('Changes vs percentile:')
print('Ustun:',np.mean(lp_rf_time_pc))
print('Brute force:',np.mean(brute_rf_time_pc))
print('Proposed:', np.mean(mapocam_rf_time_pc))
print('Proposed plus:', np.mean(mapocam_rf_time_pc_plus))
print('Features:')
print('Ustun:',np.mean(lp_rf_time_feat))
print('Brute force:',np.mean(brute_rf_time_feat))
print('Proposed:', np.mean(mapocam_rf_time_feat))
print('Proposed plus:', np.mean(mapocam_rf_time_feat_plus))

Changes vs percentile:
Ustun: 0.02952351683957709
Brute force: 0.6416896438846985
Proposed: 0.10299951235453288
Proposed plus: 0.008069576974958182
Features:
Ustun: 0.028065115203046137
Brute force: 0.4782047877709071
Proposed: 0.27347248534982405
Proposed plus: 0.01578041497204039


## Monotone trees

In [52]:
from sklearn import tree, ensemble
import lightgbm
from cfmining.predictors import MonotoneTree

In [53]:
monotone_constraints = [action.step_direction for action in action_set_base]
clf_mt = lightgbm.LGBMClassifier(n_estimators=10, max_depth=10, num_leaves=63, monotone_constraints=monotone_constraints)#, class_weight='balanced')
clf_mt.fit(Xtr, ytr)

LGBMClassifier(max_depth=10,
               monotone_constraints=[-1, -1, 1, -1, -1, 1, 1, 1, 1, 1],
               n_estimators=10, num_leaves=63)

## Preparing counterfactual parameters

In [54]:
action_set_mtree = copy.deepcopy(action_set_base)
action_set_mtree.embed_forest(clf_mt)
for action in action_set_mtree:
    action.flip_direction = action.step_direction

print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_mtree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_mtree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_mtree]))

ActionSet stats
Number of actionable features: 7
Mean number of actions per feature: 14.8
Max number of actions per feature: 56


In [55]:
scores = pd.Series(clf_mt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < .8].index

In [56]:
percCalc = PercentileCalculator(action_set=action_set_mtree)

In [57]:
mapocam_clf_mt = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=0.8)
mapocam_clf_mt_fast = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=0.8, use_predict_max=True)
mapocam_clf_mt_mon = MonotoneTree(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=0.8, use_predict_max=True)

## Linear programming

In [58]:
lp_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=0.8, max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_mt_results[i] = {}
    lp_mt_results[i]['enum'] = lp
    lp_mt_results[i]['indiv'] = individual
    lp_mt_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_mt_results[i]['time'] = lp_time
    lp_mt_results[i]['cost'] = criteria.f(lp_mt_results[i]['solution']) if not any(np.isnan(lp_mt_results[i]['solution'])) else float('inf')

100%|██████████| 100/100 [00:02<00:00, 35.05it/s]


## Greedy approach

In [59]:
greedy_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    greedy = Greedy(action_set_mtree, individual, mapocam_clf_mt, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_mt_results[i] = {}
    greedy_mt_results[i]['solution'] = greedy.solution
    greedy_mt_results[i]['time'] = greedy_time
    greedy_mt_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

100%|██████████| 100/100 [00:07<00:00, 12.67it/s]


## MAPOCAME

In [60]:
mapocam_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results[i] = {}
    mapocam_mt_results[i]['enum'] = mapocam
    mapocam_mt_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results[i]['time'] = mapocam_time
    mapocam_mt_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 100/100 [13:20<00:00,  8.00s/it]


In [61]:
mapocam_mt_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_plus[i] = {}
    mapocam_mt_results_plus[i]['enum'] = mapocam
    mapocam_mt_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_plus[i]['time'] = mapocam_time
    mapocam_mt_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 100/100 [09:51<00:00,  5.92s/it]


In [62]:
mapocam_mt_results_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_mon[i] = {}
    mapocam_mt_results_mon[i]['enum'] = mapocam
    mapocam_mt_results_mon[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_mon[i]['time'] = mapocam_time
    mapocam_mt_results_mon[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 100/100 [00:01<00:00, 88.82it/s]


In [63]:
lp_mt_costs = np.array([lp_mt_results[i]['cost'] for i in denied_individuals])
greedy_mt_costs = np.array([greedy_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs = np.array([mapocam_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_plus = np.array([mapocam_mt_results_plus[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_mon = np.array([mapocam_mt_results_mon[i]['cost'] for i in denied_individuals])

lp_mt_time = np.array([lp_mt_results[i]['time'] for i in denied_individuals])
greedy_mt_time = np.array([greedy_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time = np.array([mapocam_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time_plus = np.array([mapocam_mt_results_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_mon = np.array([mapocam_mt_results_mon[i]['time'] for i in denied_individuals])

In [64]:
print('Time performance')
print('Ustun:', np.mean(lp_mt_time))
print('Greedy:', np.mean(greedy_mt_time))
print('Proposed:', np.mean(mapocam_mt_time))
print('Proposed plus:', np.mean(mapocam_mt_time_plus))
print('Proposed mon:', np.mean(mapocam_mt_time_mon))

Time performance
Ustun: 0.022338847862556577
Greedy: 0.07834024220705033
Proposed: 8.00065245365724
Proposed plus: 5.914357251860201
Proposed mon: 0.011011128444224597


In [65]:
print('Best solution rate')
#best_costs = np.min([lp_mt_costs, greedy_mt_costs, mapocam_mt_costs, mapocam_mt_costs_plus], axis=0)
best_costs = np.min([lp_mt_costs, mapocam_mt_costs, mapocam_mt_costs_plus], axis=0)
print('Ustun:', np.mean(lp_mt_costs==best_costs))
print('Greedy:', np.mean(greedy_mt_costs==best_costs))
print('Proposed:', np.mean(mapocam_mt_costs==best_costs))
print('Proposed plus:', np.mean(mapocam_mt_costs_plus==best_costs))
print('Proposed mon:', np.mean(mapocam_mt_costs_mon==best_costs))

Best solution rate
Ustun: 1.0
Greedy: 1.0
Proposed: 1.0
Proposed plus: 1.0
Proposed mon: 1.0


## Comparing Pareto enumeration

## Percentile vs changes

In [66]:
mapocam_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc[i] = {}
    mapocam_mt_results_pc[i]['enum'] = mapocam
    mapocam_mt_results_pc[i]['time'] = mapocam_time
    mapocam_mt_results_pc[i]['num'] = len(mapocam.solutions)

100%|██████████| 100/100 [00:41<00:00,  2.43it/s]


In [67]:
mapocam_mt_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_plus[i] = {}
    mapocam_mt_results_pc_plus[i]['enum'] = mapocam
    mapocam_mt_results_pc_plus[i]['time'] = mapocam_time
    mapocam_mt_results_pc_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 100/100 [00:06<00:00, 15.49it/s]


In [68]:
mapocam_mt_results_pc_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_mon[i] = {}
    mapocam_mt_results_pc_mon[i]['enum'] = mapocam
    mapocam_mt_results_pc_mon[i]['time'] = mapocam_time
    mapocam_mt_results_pc_mon[i]['num'] = len(mapocam.solutions)

100%|██████████| 100/100 [00:01<00:00, 87.23it/s]


In [69]:
lp_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=0.8,
                               max_changes=3, multi_solution=True,
                               cost_type='linear', compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_pc[i] = {}
    lp_mt_results_pc[i]['enum'] = lp
    lp_mt_results_pc[i]['time'] = lp_time
    lp_mt_results_pc[i]['num'] = len(lp.solutions)

100%|██████████| 100/100 [00:02<00:00, 44.54it/s]


In [70]:
bf_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_pc[i] = {}
    bf_mt_results_pc[i]['enum'] = bf
    bf_mt_results_pc[i]['time'] = bf_time
    bf_mt_results_pc[i]['num'] = len(bf.solutions)

100%|██████████| 100/100 [00:40<00:00,  2.45it/s]


## Features

In [71]:
mapocam_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat[i] = {}
    mapocam_mt_results_feat[i]['enum'] = mapocam
    mapocam_mt_results_feat[i]['time'] = mapocam_time
    mapocam_mt_results_feat[i]['num'] = len(mapocam.solutions)

100%|██████████| 100/100 [00:41<00:00,  2.41it/s]


In [72]:
mapocam_mt_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_plus[i] = {}
    mapocam_mt_results_feat_plus[i]['enum'] = mapocam
    mapocam_mt_results_feat_plus[i]['time'] = mapocam_time
    mapocam_mt_results_feat_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 100/100 [00:06<00:00, 15.31it/s]


In [73]:
mapocam_mt_results_feat_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_mon[i] = {}
    mapocam_mt_results_feat_mon[i]['enum'] = mapocam
    mapocam_mt_results_feat_mon[i]['time'] = mapocam_time
    mapocam_mt_results_feat_mon[i]['num'] = len(mapocam.solutions)

100%|██████████| 100/100 [00:01<00:00, 87.00it/s]


In [74]:
lp_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=0.8,
                               max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_feat[i] = {}
    lp_mt_results_feat[i]['enum'] = lp
    lp_mt_results_feat[i]['time'] = lp_time
    lp_mt_results_feat[i]['num'] = len(lp.solutions)

100%|██████████| 100/100 [00:02<00:00, 44.41it/s]


In [75]:
bf_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_feat[i] = {}
    bf_mt_results_feat[i]['enum'] = bf
    bf_mt_results_feat[i]['time'] = bf_time
    bf_mt_results_feat[i]['num'] = len(bf.solutions)

100%|██████████| 100/100 [00:41<00:00,  2.43it/s]


In [76]:
brute_mt_sols_pc = np.array([bf_mt_results_pc[i]['num'] for i in denied_individuals])
lp_mt_sols_pc = np.array([lp_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc = np.array([mapocam_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['num'] for i in denied_individuals])

brute_mt_sols_feat = np.array([bf_mt_results_feat[i]['num'] for i in denied_individuals])
lp_mt_sols_feat = np.array([lp_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat = np.array([mapocam_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['num'] for i in denied_individuals])

brute_mt_time_pc = np.array([bf_mt_results_pc[i]['time'] for i in denied_individuals])
lp_mt_time_pc = np.array([lp_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc = np.array([mapocam_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['time'] for i in denied_individuals])

brute_mt_time_feat = np.array([bf_mt_results_feat[i]['time'] for i in denied_individuals])
lp_mt_time_feat = np.array([lp_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat = np.array([mapocam_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['time'] for i in denied_individuals])

In [77]:
print('Changes vs percentile:', np.mean(brute_mt_sols_pc==mapocam_mt_sols_pc)==1, np.mean(lp_mt_sols_pc==mapocam_mt_sols_pc)==1, np.mean(mapocam_mt_sols_pc_plus==mapocam_mt_sols_pc)==1)
print('Features:', np.mean(brute_mt_sols_feat==mapocam_mt_sols_feat)==1, np.mean(lp_mt_sols_feat==mapocam_mt_sols_feat)==1, np.mean(mapocam_mt_sols_feat_plus==mapocam_mt_sols_feat)==1)

Changes vs percentile: True True True
Features: True True True


In [78]:
print('Changes vs percentile:')
print('Ustun:',np.mean(lp_mt_time_pc))
print('Brute force:',np.mean(brute_mt_time_pc))
print('Proposed:', np.mean(mapocam_mt_time_pc))
print('Proposed plus:', np.mean(mapocam_mt_time_pc_plus))
print('Proposed mon:', np.mean(mapocam_mt_time_pc_mon))
print('Features:')
print('Ustun:',np.mean(lp_mt_time_feat))
print('Brute force:',np.mean(brute_mt_time_feat))
print('Proposed:', np.mean(mapocam_mt_time_feat))
print('Proposed plus:', np.mean(mapocam_mt_time_feat_plus))
print('Proposed mon:', np.mean(mapocam_mt_time_feat_mon))

Changes vs percentile:
Ustun: 0.022091966969892383
Brute force: 0.4074050876032561
Proposed: 0.4106127476319671
Proposed plus: 0.06400029682554305
Proposed mon: 0.01122420541010797
Features:
Ustun: 0.022159120552241802
Brute force: 0.41080246372148393
Proposed: 0.4130179724562913
Proposed plus: 0.064749336829409
Proposed mon: 0.011253681853413583
